In [54]:
import sys
sys.path.append('..')
from core import graph

import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Construct a simple pipeline

In [99]:
@graph.optex_process('start_dataset')
def load_dataset(name):
    return pd.DataFrame({'a': [1,2,3], 'b': [6,3,4]})


@graph.optex_process('dataset_two')
def transform_one(first_dataset):
    return first_dataset + 10


@graph.optex_process('dataset_three')
def transform_two(second_dataset):
    return second_dataset / 10


@graph.optex_composition('final_dataset')
def process_dataset(process_dataset):
    d1 = transform_one(process_dataset)
    d2 = transform_two(d1)
    return d2

d = process_dataset(load_dataset(graph.Artifact('ahhh')))

SCOPE:  None


In [100]:
d.parents['final_dataset'].parents['process_dataset'].parents['start_dataset'].parents

{'name': <core.graph.Artifact at 0x7f08e2fbe550>}

In [101]:
d.parents['final_dataset'].child_processes[0].children['dataset_two'].children['second_dataset'].children['dataset_three'].parents

{'final_dataset': <core.graph.Process at 0x7f08e2fae0a0>}

## A more complicated pipeline

In [103]:
@graph.optex_process('transform_return')
def transform(transform_arg):
    return transform_arg + 10


@graph.optex_composition('inner_return')
def inner_compose(inner_arg):
    return transform(inner_arg)


@graph.optex_composition('outer_return')
def outer_compose(outer_arg):
    return inner_compose(outer_arg)

d = outer_compose(graph.Artifact(20))

In [128]:
d.parents['outer_return'].child_processes[0].child_processes[0].children

{'transform_return': <core.graph.Artifact at 0x7f08e2fa8700>}